In [ ]:
# 1. Kaggle API 설치 및 설정
!pip install -q kaggle
!mkdir -p ~/.kaggle

# 본인의 kaggle.json 파일을 /content/kaggle.json 경로에 업로드 후 실행하세요.
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# 2. 데이터셋 다운로드 및 압축 해제
!kaggle datasets download -d hasnainjaved/melanoma-skin-cancer-dataset-of-10000-images
!unzip -q melanoma-skin-cancer-dataset-of-10000-images.zip

Dataset URL: https://www.kaggle.com/datasets/hasnainjaved/melanoma-skin-cancer-dataset-of-10000-images
License(s): CC0-1.0
  0% 0.00/98.7M [00:00<?, ?B/s]
100% 98.7M/98.7M [00:00<00:00, 1.05GB/s]


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision.transforms as transforms

import os
import cv2
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# 장치 설정 (GPU 우선 사용)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

# 데이터셋 경로
DATASET_PATH = 'melanoma_cancer_dataset'

Using device: cuda


In [ ]:
# 이미지 변환 정의
# Keras의 preprocess_input과 resize를 대체합니다.
# RegNet 모델이 ImageNet으로 사전 훈련될 때 사용된 정규화 값입니다.
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((320, 320)),
        transforms.RandomHorizontalFlip(), # 데이터 증강
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((320, 320)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

class MelanomaDataset(Dataset):
    def __init__(self, root_dir, mode='train', transform=None):
        self.meta = []
        self.transform = transform

        # Keras 코드의 os.walk 부분을 재구성하여 파일 경로와 레이블 리스트를 생성합니다.
        data_path = os.path.join(root_dir, 'train' if mode == 'train' else 'test')
        for label, category in enumerate(['benign', 'malignant']):
            class_path = os.path.join(data_path, category)
            for filename in os.listdir(class_path):
                if filename.endswith('.jpg'):
                    path = os.path.join(class_path, filename)
                    self.meta.append((path, label))

    def __len__(self):
        return len(self.meta)

    def __getitem__(self, idx):
        path, label = self.meta[idx]
        # Keras의 load_image 함수 부분을 대체합니다.
        image = Image.open(path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(label, dtype=torch.float32)

# 데이터셋 인스턴스 생성
train_dataset = MelanomaDataset(root_dir=DATASET_PATH, mode='train', transform=data_transforms['train'])
valid_dataset = MelanomaDataset(root_dir=DATASET_PATH, mode='val', transform=data_transforms['val'])

# 데이터로더 생성
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
valid_loader = DataLoader(valid_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

# 데이터셋 크기 확인
print(f'Train dataset size: {len(train_dataset)}')
print(f'Validation dataset size: {len(valid_dataset)}')

# 데이터로더 테스트
images, labels = next(iter(train_loader))
print(f'Image batch shape: {images.shape}')
print(f'Label batch shape: {labels.shape}')

Train dataset size: 9605
Validation dataset size: 1000


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Image batch shape: torch.Size([32, 3, 320, 320])
Label batch shape: torch.Size([32])


In [ ]:
# Keras의 base_model에 해당합니다. ImageNet으로 사전 훈련된 가중치를 사용합니다.
model = models.regnet_x_3_2gf(pretrained=True)

# Keras의 GlobalAvgPool2D + Dense(1, activation='sigmoid') 부분을 대체합니다.
# RegNet의 마지막 fully-connected layer를 이진 분류에 맞게 교체합니다.
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 1) # 출력 1개

model = model.to(device)

# 모델 구조 확인 (Keras의 summary와 유사)
# print(model)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=RegNet_X_3_2GF_Weights.IMAGENET1K_V1`. You can also use `weights=RegNet_X_3_2GF_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/regnet_x_3_2gf-f342aeae.pth" to /root/.cache/torch/hub/checkpoints/regnet_x_3_2gf-f342aeae.pth
100%|██████████| 58.8M/58.8M [00:01<00:00, 46.5MB/s]


In [ ]:
def train_model(model, criterion, optimizer, train_loader, valid_loader, epochs):
    for epoch in range(epochs):
        # --- 훈련 단계 ---
        model.train()
        running_loss = 0.0
        correct_predictions = 0
        total_samples = 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

            preds = torch.sigmoid(outputs) > 0.5
            correct_predictions += (preds == labels).sum().item()
            total_samples += labels.size(0)

        epoch_loss = running_loss / total_samples
        epoch_acc = correct_predictions / total_samples

        # --- 검증 단계 ---
        model.eval()
        val_loss = 0.0
        val_corrects = 0
        val_total = 0

        with torch.no_grad():
            for inputs, labels in valid_loader:
                inputs, labels = inputs.to(device), labels.to(device).view(-1, 1)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * inputs.size(0)
                preds = torch.sigmoid(outputs) > 0.5
                val_corrects += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_epoch_loss = val_loss / val_total
        val_epoch_acc = val_corrects / val_total

        print(f"Epoch {epoch+1}/{epochs} - "
              f"loss: {epoch_loss:.4f} - accuracy: {epoch_acc:.4f} - "
              f"val_loss: {val_epoch_loss:.4f} - val_accuracy: {val_epoch_acc:.4f}")

# 손실 함수와 옵티마이저 정의
criterion = nn.BCEWithLogitsLoss()

# --- 1단계: 전이 학습 (Feature Extraction) ---
# Keras의 `base_model.trainable = False`에 해당
for param in model.parameters():
    param.requires_grad = False
# 새로 추가한 fc 레이어의 파라미터만 학습하도록 설정
for param in model.fc.parameters():
    param.requires_grad = True

# fc 레이어의 파라미터만 optimizer에 전달
optimizer_transfer = optim.Adam(model.fc.parameters(), lr=0.001)

print("--- Start Transfer Learning (Training Head) ---")
# 5 에포크 훈련
train_model(model, criterion, optimizer_transfer, train_loader, valid_loader, epochs=5)


# --- 2단계: 미세 조정 (Fine-tuning) ---
# Keras의 `base_model.trainable = True`에 해당 (문서에서는 모든 가중치의 프리즈를 푼다고 언급)
for param in model.parameters():
    param.requires_grad = True

# 전체 모델의 파라미터를 optimizer에 전달. 보통 더 낮은 학습률(learning rate)을 사용합니다.
optimizer_finetune = optim.Adam(model.parameters(), lr=0.0001)

print("\n--- Start Fine-tuning (Training All Layers) ---")
# 15 에포크 추가 훈련
train_model(model, criterion, optimizer_finetune, train_loader, valid_loader, epochs=15)

--- Start Transfer Learning (Training Head) ---
Epoch 1/5 - loss: 0.3318 - accuracy: 0.8612 - val_loss: 0.2356 - val_accuracy: 0.9070
Epoch 2/5 - loss: 0.2540 - accuracy: 0.8970 - val_loss: 0.2299 - val_accuracy: 0.9090
Epoch 3/5 - loss: 0.2436 - accuracy: 0.9017 - val_loss: 0.2268 - val_accuracy: 0.9080
Epoch 4/5 - loss: 0.2365 - accuracy: 0.9045 - val_loss: 0.2430 - val_accuracy: 0.9060
Epoch 5/5 - loss: 0.2230 - accuracy: 0.9107 - val_loss: 0.2599 - val_accuracy: 0.8990

--- Start Fine-tuning (Training All Layers) ---
Epoch 1/15 - loss: 0.1991 - accuracy: 0.9208 - val_loss: 0.2005 - val_accuracy: 0.9150
Epoch 2/15 - loss: 0.1246 - accuracy: 0.9503 - val_loss: 0.1975 - val_accuracy: 0.9230
Epoch 3/15 - loss: 0.0887 - accuracy: 0.9678 - val_loss: 0.2165 - val_accuracy: 0.9210
Epoch 4/15 - loss: 0.0629 - accuracy: 0.9771 - val_loss: 0.2543 - val_accuracy: 0.9240
Epoch 5/15 - loss: 0.0483 - accuracy: 0.9826 - val_loss: 0.2703 - val_accuracy: 0.9320
Epoch 6/15 - loss: 0.0426 - accuracy: 